In [1]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn import svm
import math
import tensorflow as tf

In [2]:
class RandomAgent:
    def __init__(self, env, num_experiments=10, num_episodes=100, num_timesteps=1000):
        self.env = env
        self.num_experiments=num_experiments
        self.num_episodes=num_episodes
        self.num_timesteps=num_timesteps
    def getAction(self, state):
        return self.env.action_space.sample()
    def getQValue(self, state):
        pass
    def update(self, state, action, reward, nextState):
        pass
    def run(self):
        episodic_return = np.zeros((self.num_experiments, self.num_episodes))
        for i in range(self.num_experiments):
            for j in range(self.num_episodes):
                acc_reward = 0
                for t in range(self.num_timesteps):
                    observation = self.env.reset()
                    action = self.getAction(observation)
                    nextObservation, reward, done, info = self.env.step(action)
                    if done:
                        break
                    acc_reward += reward
                episodic_return[i, j] += acc_reward
        return episodic_return

In [55]:

class PGAgent:
    def __init__(self, env, num_experiments=10, num_episodes=100, num_timesteps=1000, replay_buffer_size=10000, epsilon=0.05):
        self.env = env
        self.num_experiments=num_experiments
        self.num_episodes=num_episodes
        self.num_timesteps=num_timesteps
        self.num_actions = 2
        self.replay_buffer_size = replay_buffer_size
        self.observation_size = self.env.observation_space.shape[0]
        self.epsilon = epsilon
        self.replay_buffer = []
        self.optim = tf.train.AdamOptimizer(learning_rate=0.001)
        
        self.state = tf.placeholder(tf.float32, [self.observation_size])
        self.next_state = tf.placeholder(tf.float32, [self.observation_size])
        self.reward = tf.placeholder(tf.float32)
        self.action = tf.placeholder(tf.int32)
        
        def network(input_data, scope):
            with tf.variable_scope(scope):
                w1 = tf.layers.dense(input_data, 16,  name="input_layer")
                w2 = tf.layers.dense(w1, 32, name="h1")
                out = tf.layers.dense(w2, self.num_actions, name="h2")
            return out
        
        self.Q_network = network(self.state, "network")
        self.Q_target = network(self.next_state, "target")
        

        
        
        
        network_params = [t for t in tf.trainable_variables() if t.name.startswith("network")]
        network_params = sorted(network_params, key=lambda v: v.name)
        target_params = [t for t in tf.trainable_variables() if t.name.startswith("target")]
        target_params = sorted(target_params, key=lambda v: v.name)

        update_target = []
        for network_v, target_v in zip(network_params, target_params):
            op = target_v.assign(network_v)
            update_target.append(op)


                
        self.update_target = update_target
        
        self.loss = 1/2*tf.pow(self.reward + tf.reduce_max(self.Q_target)- self.Q_network[self.action], 2)
        self.session = tf.Session()
        self.session.run(tf.initialize_all_variables())
        
        
        # self.alpha = 0.2
        # self.beta = 0.2
    
        # Your code here!
        # Features can for example come from a grid of Radial Basis Functions
        # or some other model that lets you do predictions over unvisited states
        # self.features = ?
        
        ### You can store parts of the state space like this and build your kernels
        #
        # self.observations = [[] for i in range(num_action)]
        # for i in self.observations:
        #     i.append(self.env.observation_space.sample())
        #
        ###
        
        ### You need to store your weights somewhere
        ### if you are using actor-critic you probably want weights for both the policy and the value function!
        #
        # self.PIweights = np.ones((num_action,?))
        # self.Vweights = np.ones((num_action,?))
        #
        ###
        
    ### This method will return a sampled action
    ### from your policy
    def getAction(self, state, epsilon=0.05):
        Q = self.getQValues(state)
        if random.random() < epsilon:
            return random.choice(range(self.num_actions))
        else:
            return np.argmax(Q)
        
    def getQValues(self, state):
        return self.session.run(self.Q_network, feed_dict= {self.state: state})
    
                
    def update_target(self):
        self.session.run(self.update_target)
        
    def update(self):
        state, action, reward, next_state = self.sample_from_buffer()
        self.session.run(self.optim.minimize(self.loss), feed_dict={
                self.state: state, self.action: action, self.reward: reward, self.next_state: next_state}
                )
       
    def add_replay_buffer(self,x):
        self.replay_buffer.append(x)
        if len(self.replay_buffer)>self.replay_buffer_size:
            self.replay_buffer = self.replay_buffer[-self.replay_buffer_size:]
            
    def sample_from_buffer(self):
        buffer_length = len(self.replay_buffer)
        sample = np.random.choice(buffer_length)
        return self.replay_buffer[sample]
        
            
    def run(self):
        episodic_return = np.zeros((self.num_experiments, self.num_episodes))
        for i in range(self.num_experiments):
            # first collect some MC returns 
            for j in range(self.num_episodes):
                acc_reward = 0
                observation = self.env.reset()
                for t in range(self.num_timesteps):
                    action = self.getAction(observation)
                    nextObservation, reward, done, info = self.env.step(action)
                    self.add_replay_buffer((observation, action, reward, nextObservation))
                    acc_reward += reward
                    self.update()
                    if done:
                        break
 
                    observation = nextObservation
                episodic_return[i, j] += acc_reward
                print(acc_reward)
        return episodic_return
    

In [54]:
# pg agent
tf.reset_default_graph()
env = gym.make("CartPole-v0")
ragent = PGAgent(env, num_episodes=1000, num_experiments=2)
eps_return = ragent.run()
print (eps_return.shape)
print (np.mean(eps_return, axis=0))
plt.scatter()

# random agent
env = gym.make("CartPole-v0")
ragent = RandomAgent(env)
eps_return = ragent.run()
print( np.mean(eps_return, axis=0))

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


ValueError: Input 0 of layer input_layer is incompatible with the layer: : expected min_ndim=2, found ndim=1. Full shape received: [4]